#**Knowledge and Data Mining Project**


---
#**Number Mind**
**PROF.LUCIANO SERAFINI**

**Berke Furkan Kusmenoglu** 

**2041496**

**berkefurkan.kusmenoglu@studenti.unipd.it**

In [19]:
 !pip install python-sat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import numpy as np
from pysat.solvers import Minisat22
from time import process_time
from random import randint
from itertools import combinations


In [21]:
def get_unique_integer(n: int, p: int, size: int) -> int:
    return n * size + p + 1

for n in range(10):
    for p in range(5):
        print(f"get_unique_integer({n}, {p}, 5) ----> {get_unique_integer(n, p, 5)}")


get_unique_integer(0, 0, 5) ----> 1
get_unique_integer(0, 1, 5) ----> 2
get_unique_integer(0, 2, 5) ----> 3
get_unique_integer(0, 3, 5) ----> 4
get_unique_integer(0, 4, 5) ----> 5
get_unique_integer(1, 0, 5) ----> 6
get_unique_integer(1, 1, 5) ----> 7
get_unique_integer(1, 2, 5) ----> 8
get_unique_integer(1, 3, 5) ----> 9
get_unique_integer(1, 4, 5) ----> 10
get_unique_integer(2, 0, 5) ----> 11
get_unique_integer(2, 1, 5) ----> 12
get_unique_integer(2, 2, 5) ----> 13
get_unique_integer(2, 3, 5) ----> 14
get_unique_integer(2, 4, 5) ----> 15
get_unique_integer(3, 0, 5) ----> 16
get_unique_integer(3, 1, 5) ----> 17
get_unique_integer(3, 2, 5) ----> 18
get_unique_integer(3, 3, 5) ----> 19
get_unique_integer(3, 4, 5) ----> 20
get_unique_integer(4, 0, 5) ----> 21
get_unique_integer(4, 1, 5) ----> 22
get_unique_integer(4, 2, 5) ----> 23
get_unique_integer(4, 3, 5) ----> 24
get_unique_integer(4, 4, 5) ----> 25
get_unique_integer(5, 0, 5) ----> 26
get_unique_integer(5, 1, 5) ----> 27
get_unique

In [22]:
def encode(encoded_guess):
    return [get_unique_integer(int(x),i,len(encoded_guess)) for i,x in enumerate(encoded_guess)]

def decode(encoded_guess):
    size = len(encoded_guess)
    decoded_guess = [0] * size
    for n in encoded_guess:
        for p in range(size):
            if (n - 1 - p) % len(encoded_guess) == 0:
                decoded_guess[p] = (n - 1 - p)/size
                break
    decoded_guess = ''.join(str(int(x)) for x in decoded_guess)
    return decoded_guess

def check_right(encoded_guess, encoded_code):
    count = 0
    for guess_num in encoded_guess:
        if guess_num in encoded_code:
            count += 1
    return count

def append_clauses(encoded_guess, num_correct, s):
    if num_correct == 0:
        for x in encoded_guess:
            s.add_clause([-int(x)])
    else:
        least_k = combinations(encoded_guess, len(encoded_guess) - num_correct + 1)
        most_k = combinations(encoded_guess, num_correct + 1)
        
        for clause in least_k:
            s.add_clause([int(x) for x in clause])
        
        for clause in most_k:
            s.add_clause([-int(x) for x in clause])
        
    return s

In [23]:
def NumberMind():
    start_time = process_time()
    correct_num = 0
    stop = False
    digit = 5
    
    # Generate a secret code
    secret_code = np.random.randint(0, 10, digit)
    print(f"Secret Code : {int(''.join(map(str, secret_code)))}")
    print("##############################################")
    
    # Encode the secret code and initial guess
    encoded_code = [get_unique_integer(int(x), i, digit) for i, x in enumerate(secret_code)]
    initial_guess = np.random.randint(0, 10, digit)
    encoded_guess = [get_unique_integer(int(x), i, digit) for i, x in enumerate(initial_guess)]

    # Initialize a solver
    solver = Minisat22()

    # Add constraints for exactly one number in each position
    for row in range(digit):
        pos_vars = [get_unique_integer(n, row, digit) for n in range(10)]
        solver.add_clause(pos_vars)

    # Add constraints for at most one number in each position
    for row in range(digit):
        for n in range(10):
            for n1 in range(n+1, 10):
                solver.add_clause([-get_unique_integer(n, row, digit), -get_unique_integer(n1, row, digit)])

    #Round-1
    counter = 1; 
    # Guess the secret code
    while not stop:
        new_guess = []
        number_of_right = check_right(encoded_guess, encoded_code)  # Count the right digits in the guess
        
        # Check if the guess is right
        print("You are in the:" ,counter, "round")
        print(f"Guess is: {decode(encoded_guess)}")
        print("The number of correct digits:" ,number_of_right)
        print("##############################################")
        if number_of_right == digit:
            stop = True
            
        # Add clauses based on the guess
        solver = append_clauses(encoded_guess, number_of_right, solver)
        solver.solve()
                
        # Get the new guess from the model
        for model in solver.enum_models():
            new_guess = [x for x in model if x > 0]
            break
        
        # Update the guess
        encoded_guess = new_guess
        counter = counter + 1
    end_time = process_time()
    print(f"CPU time: {end_time - start_time}")

In [24]:
NumberMind()

Secret Code : 29961
##############################################
You are in the: 1 round
Guess is: 38588
The number of correct digits: 0
##############################################
You are in the: 2 round
Guess is: 00000
The number of correct digits: 0
##############################################
You are in the: 3 round
Guess is: 91111
The number of correct digits: 1
##############################################
You are in the: 4 round
Guess is: 89122
The number of correct digits: 1
##############################################
You are in the: 5 round
Guess is: 97992
The number of correct digits: 1
##############################################
You are in the: 6 round
Guess is: 87819
The number of correct digits: 0
##############################################
You are in the: 7 round
Guess is: 11232
The number of correct digits: 0
##############################################
You are in the: 8 round
Guess is: 99347
The number of correct digits: 1
############################